In [3]:
import pandas as pd
import numpy as np
import requests
import json
import glob
from fastparquet import ParquetFile
import pyarrow as pa
import pyarrow.parquet as pq

test_folder_path = '../data/extracts/v18/worldwide/105670/parquet'

#SELECT sum (ground_up_loss_inc_ds) from  [OPTISIMDB_Eventsets_AIR18_Vendor].[dbo].[t105670_event_loss]
expected_loss = 917069225546531.782

In [7]:
%%timeit -r 2
print("fastparquet")

parq_files = glob.glob(test_folder_path + '/*.parquet')
fast_parquet = ParquetFile(parq_files)
fast_parquet_df = fast_parquet.to_pandas()
print(fast_parquet_df.Loss.sum())

fastparquet


MemoryError: 

In [8]:
%%timeit -r 2
print("pyarrow")

pyarrow_dataset = pq.ParquetDataset(test_folder_path)
pyarrow_table = pyarrow_dataset.read()
pyarrow_df = pyarrow_table.to_pandas()
print(pyarrow_df.Loss.sum())

pyarrow


ArrowIOError: Arrow error: Out of memory: malloc of size 33554432 failed

In [5]:
%%timeit -r 2
print("pyspark")

spark_df = sqlContext.read.load(test_folder_path)
spark_df.groupBy().sum('Loss').show()

+-------------------+
|          sum(Loss)|
+-------------------+
|9.14031158746916E14|
+-------------------+

+-------------------+
|          sum(Loss)|
+-------------------+
|9.14031158746916E14|
+-------------------+

+-------------------+
|          sum(Loss)|
+-------------------+
|9.14031158746916E14|
+-------------------+

1.03 s ± 94.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
